<a href="https://colab.research.google.com/github/Veleces/Project/blob/main/Fundamental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading


First, we need to mount this notebook to our Google Drive folder, in order to access the CSV data file.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


And now we can import this package like so:

In [2]:
import pandas as pd
import numpy as np

Now that Google Drive is mounted, you can store the CSV file anywhere in your Drive and update the path variable below to access it within this notebook. Once we've updated the path, let's read this CSV file into a pandas dataframe and see what it looks like

In [3]:
path = "/content/drive/MyDrive/Holdings/"

fundamental_analysis_data = pd.read_csv(f"{path}IWV_holdings_Black Rock.csv", encoding='latin-1')
fundamental_analysis_data.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
fundamental_analysis_data.head()

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Renta Variable,829862857.4,6.09,829862857.4,2012667,412.32,Estados Unidos,NASDAQ,USD,1,USD,-
1,AAPL,APPLE INC,Tecnología de la Información,Renta Variable,727953308.5,5.34,727953308.5,3944050,184.57,Estados Unidos,NASDAQ,USD,1,USD,-
2,NVDA,NVIDIA CORP,Tecnología de la Información,Renta Variable,570283784.7,4.19,570283784.7,642595,887.47,Estados Unidos,NASDAQ,USD,1,USD,-
3,AMZN,AMAZON COM INC,Consumo discrecional,Renta Variable,461714476.0,3.39,461714476.0,2436488,189.50,Estados Unidos,NASDAQ,USD,1,USD,-
4,META,META PLATFORMS INC CLASS A,Comunicación,Renta Variable,284383407.7,2.09,284383407.7,598173,475.42,Estados Unidos,NASDAQ,USD,1,USD,-


In [4]:
# Selecting only the specified columns
selected_columns = ['Ticker', 'Name', 'Sector']
fundamental_analysis_data = fundamental_analysis_data[selected_columns]

# Display the first few rows of the subset DataFrame
fundamental_analysis_data


,Ticker,Name,Sector
0,MSFT,MICROSOFT CORP,Tecnología de la Información
1,AAPL,APPLE INC,Tecnología de la Información
2,NVDA,NVIDIA CORP,Tecnología de la Información
3,AMZN,AMAZON COM INC,Consumo discrecional
4,META,META PLATFORMS INC CLASS A,Comunicación
...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud
2683,P5N994,Petrocorp Inc Escrow,Energía
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez


**Install the required libraries**

In [5]:
!pip install -q yfinance
!pip install mplfinance
!pip install --upgrade mplfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 798.2 kB/s eta 0:00:00


In [6]:
# Get the tickers from the DataFrame
tickers = fundamental_analysis_data["Ticker"].tolist()

Functions to collect Yahho Finance Data

In [7]:
# Create a dictionary to map the market cap classification
market_cap_classification = {
    "Mega": "Mega Cap",
    "Large": "Large Cap",
    "Big": "Big Cap",
    "Mid": "Mid Cap",
    "Small-Mid": "Small-Mid Cap",
    "Small": "Small Cap",
    "Micro": "Micro Cap",
    "Nano": "Nano Cap"
}

# Function to get the market cap and its classification
def get_market_cap_classification(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        market_cap = ticker_info.info.get('marketCap', None)
        if market_cap is not None:
            if market_cap >= 1e12:  # Mega Cap
                return "Mega", market_cap
            elif 200e9 <= market_cap < 1e12:  # Large Cap
                return "Large", market_cap
            elif 100e9 <= market_cap < 200e9:  # Big Cap
                return "Big", market_cap
            elif 10e9 <= market_cap < 100e9:  # Mid Cap
                return "Mid", market_cap
            elif 2e9 <= market_cap < 10e9:  # Small-Mid Cap
                return "Small-Mid", market_cap
            elif 300e6 <= market_cap < 2e9:  # Small Cap
                return "Small", market_cap
            elif 50e6 <= market_cap < 300e6:  # Micro Cap
                return "Micro", market_cap
            else:  # Nano Cap
                return "Nano", market_cap
        else:
            return "N/A", "N/A"
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A", "N/A"

# Function to get total debt and operating cash flow
def get_debt_and_cash_flow(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get total debt
        total_debt = company_info.info.get('totalDebt', "N/A")

        # Get operating cash flow
        operating_cash_flow = company_info.info.get('operatingCashflow', "N/A")

        return total_debt, operating_cash_flow
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A", "N/A"

# Function to fetch trailing P/E ratio
def get_trailing_pe(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get trailing P/E ratio
        trailing_pe = company_info.info.get('trailingPE', np.nan)

        # Convert '--' to NaN
        if trailing_pe == 'Infinity':
            trailing_pe = np.nan
        else:
            trailing_pe = float(trailing_pe)

        return trailing_pe
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

# Function to fetch forward P/E ratio
def get_forward_pe(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get forward P/E ratio
        forward_pe = company_info.info.get('forwardPE', np.nan)

        # Convert '--' to NaN
        if forward_pe == 'Infinity':
            forward_pe = np.nan
        else:
            forward_pe = float(forward_pe)

        return forward_pe
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

# Function to fetch the PEG ratio
def get_peg_ratio(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get the PEG ratio
        peg_ratio = company_info.info.get('pegRatio', np.nan)

        # Convert '--' to NaN
        if peg_ratio == 'Infinity':
            peg_ratio = np.nan
        else:
            peg_ratio = float(peg_ratio)

        return peg_ratio
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

import yfinance as yf

def get_price_to_sales(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get Price/Sales ratio
        price_to_sales = company_info.info.get('priceToSalesTrailing12Months', np.nan)

        # Convert '--' to NaN
        if price_to_sales == 'Infinity':
            price_to_sales = np.nan
        else:
            price_to_sales = float(price_to_sales)

        return price_to_sales
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

def get_price_to_book(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get the Price to Book ratio
        price_to_book = company_info.info.get('priceToBook', np.nan)

        # Convert '--' to NaN
        if price_to_book == 'Infinity':
            price_to_book = np.nan
        else:
            price_to_book = float(price_to_book)

        return price_to_book
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

def get_eps(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get trailing EPS
        trailing_eps = company_info.info.get('trailingEps', "N/A")

        # Get forward EPS
        forward_eps = company_info.info.get('forwardEps', "N/A")

        # Convert '--' to NaN
        if trailing_eps == '--':
            trailing_eps = "N/A"
        else:
            trailing_eps = float(trailing_eps)

        if forward_eps == '--':
            forward_eps = "N/A"
        else:
            forward_eps = float(forward_eps)

        return trailing_eps, forward_eps
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A", "N/A"

# Function to fetch trailing EPS from Yahoo Finance
def get_trailing_eps(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get trailing EPS
        trailing_eps = company_info.info.get('trailingEps', np.nan)

        # Convert '--' to NaN
        if trailing_eps == '--':
            trailing_eps = np.nan
        else:
            trailing_eps = float(trailing_eps)

        return trailing_eps
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

# Function to fetch forward EPS from Yahoo Finance
def get_forward_eps(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get forward EPS
        forward_eps = company_info.info.get('forwardEps', np.nan)

        # Convert '--' to NaN
        if forward_eps == '--':
            forward_eps = np.nan
        else:
            forward_eps = float(forward_eps)

        return forward_eps
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan

def get_current_price(ticker):
    try:
        # Fetch information about the company
        company_info = yf.Ticker(ticker)

        # Get the current price
        current_price = company_info.info.get('currentPrice', "N/A")

        # Convert '--' to NaN
        if current_price == '--':
            current_price = "N/A"
        else:
            current_price = float(current_price)

        return current_price
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A"

def get_net_income_growth(ticker):
    try:
        # Fetch financial information about the company
        company_info = yf.Ticker(ticker)

        # Get the income statement
        income_statement = company_info.financials

        # Extract net income for the past four years
        net_income = income_statement.loc['Net Income'].iloc[:4]

        # Convert '--' values to NaN
        net_income = net_income.apply(lambda x: np.nan if x == '--' else x)

        # Convert to numeric values (just in case)
        net_income = pd.to_numeric(net_income, errors='coerce')

        # Reverse the order to have the most recent year last
        net_income = net_income[::-1]

        # Calculate the year-over-year percentage growth
        growth_rates = net_income.pct_change().dropna() * 100

        # Filter out infinite values
        growth_rates = growth_rates.replace([np.inf, -np.inf], np.nan).dropna()

        # Calculate the average growth rate
        average_growth_rate = growth_rates.mean()

        return average_growth_rate
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A"

def get_free_cash_flow_growth(ticker):
    try:
        # Fetch information about the company
        company = yf.Ticker(ticker)

        # Get the cash flow statement
        cash_flow_statement = company.cashflow

        # Extract Free Cash Flow (FCF) values
        free_cash_flow = cash_flow_statement.loc['Free Cash Flow'].iloc[:4]

        # Convert '--' values to NaN
        free_cash_flow = free_cash_flow.apply(lambda x: np.nan if x == '--' else x)

        # Convert to numeric values (just in case)
        free_cash_flow = pd.to_numeric(free_cash_flow, errors='coerce')

        # Reverse the order to have the most recent year last
        free_cash_flow = free_cash_flow[::-1]

        # Calculate the year-over-year percentage growth
        growth_rates = free_cash_flow.pct_change().dropna() * 100

        # Filter out infinite values
        growth_rates = growth_rates.replace([np.inf, -np.inf], np.nan).dropna()

        # Calculate the average growth rate
        average_growth_rate = growth_rates.mean()

        return average_growth_rate
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A"

def get_total_liabilities_growth(ticker):
    try:
        # Fetch information about the company
        company = yf.Ticker(ticker)

        # Get the balance sheet data
        balance_sheet = company.balance_sheet

        # Extract Total Liabilities Net Minority Interest for the past four years
        total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[:4]

        # Convert '--' values to NaN
        total_liabilities = total_liabilities.apply(lambda x: np.nan if x == '--' else x)

        # Convert to numeric values (just in case)
        total_liabilities = pd.to_numeric(total_liabilities, errors='coerce')

        # Reverse the order to have the most recent year last
        total_liabilities = total_liabilities[::-1]

        # Calculate the year-over-year percentage growth
        growth_rates = total_liabilities.pct_change().dropna() * 100

        # Filter out infinite values
        growth_rates = growth_rates.replace([np.inf, -np.inf], np.nan).dropna()

        # Calculate the average growth rate
        average_growth_rate = growth_rates.mean()

        return average_growth_rate
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return "N/A"

In [8]:
import yfinance as yf
import mplfinance as mpf
import numpy as np

# Apply the function to each ticker and add the results as new columns to the DataFrame
fundamental_analysis_data["Market Cap Classification"], fundamental_analysis_data["Market Cap"] = zip(*fundamental_analysis_data["Ticker"].apply(get_market_cap_classification))


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [9]:
# Apply the function to each ticker and add the total debt and operating cash flow as new columns to the DataFrame
fundamental_analysis_data["Total Debt"], fundamental_analysis_data["Operating Cash Flow"] = zip(*fundamental_analysis_data["Ticker"].apply(get_debt_and_cash_flow))


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69


Error fetching data for FCF: 'NoneType' object is not subscriptable


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=METCV&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GEFB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GEFB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/P5N994?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=P5N994&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RTYM4?modules=financialData%2CquoteType%2CdefaultK

In [10]:
# Convert the data types of Market Cap, Total Debt, and Operating Cash Flow columns to numeric
fundamental_analysis_data["Market Cap"] = pd.to_numeric(fundamental_analysis_data["Market Cap"], errors='coerce')
fundamental_analysis_data["Total Debt"] = pd.to_numeric(fundamental_analysis_data["Total Debt"], errors='coerce')
fundamental_analysis_data["Operating Cash Flow"] = pd.to_numeric(fundamental_analysis_data["Operating Cash Flow"], errors='coerce')

# Calculate Enterprise Value
fundamental_analysis_data["Enterprise Value"] = (fundamental_analysis_data["Market Cap"] + fundamental_analysis_data["Total Debt"]) - fundamental_analysis_data["Operating Cash Flow"]


In [11]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12
...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN


In [12]:


# Check for missing or invalid values in Market Cap and Enterprise Value columns
missing_values = fundamental_analysis_data[['Market Cap', 'Enterprise Value']].isnull().any(axis=1)
invalid_values = ~fundamental_analysis_data[['Market Cap', 'Enterprise Value']].apply(pd.to_numeric, errors='coerce').notna().all(axis=1)

# Filter out rows with missing or invalid values
valid_data = fundamental_analysis_data[~(missing_values | invalid_values)]

# Subtract Enterprise Value from Market Cap to get Residual Enterprise Value
fundamental_analysis_data["Residual Enterprise Value"] = valid_data["Market Cap"] - valid_data["Enterprise Value"]

# Display the DataFrame with the new column
fundamental_analysis_data.head()


,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10


In [13]:
# Calculate % Residual Enterprise Value
fundamental_analysis_data["% Residual Enterprise Value"] = (fundamental_analysis_data["Residual Enterprise Value"] / fundamental_analysis_data["Market Cap"]) * 100

# Display the DataFrame with the new column
fundamental_analysis_data


,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694
...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Sort the DataFrame by % Residual Enterprise Value in ascending order
sorted_data = fundamental_analysis_data.sort_values(by='% Residual Enterprise Value',ascending=False)

# Display the sorted DataFrame
sorted_data


,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value
1253,PAGS,PAGSEGURO DIGITAL LTD CLASS A,Financieros,Small-Mid,3.914977e+09,1.011011e+09,6.029550e+09,-1.103562e+09,5.018539e+09,128.188221
1966,FG,F&G ANNUITIES AND LIFE INC,Financieros,Small-Mid,5.127550e+09,1.778000e+09,5.931000e+09,9.745507e+08,4.153000e+09,80.993836
1960,TK,TEEKAY CORP,Energía,Small,8.946317e+08,7.995500e+07,6.084260e+08,3.661608e+08,5.284710e+08,59.071343
1927,CCRN,CROSS COUNTRY HEALTHCARE INC,Cuidado de la Salud,Small,5.042244e+08,4.492000e+06,2.076440e+08,3.010724e+08,2.031520e+08,40.289994
2331,OSUR,ORASURE TECHNOLOGIES INC,Cuidado de la Salud,Small,3.587026e+08,1.299000e+07,1.423190e+08,2.293736e+08,1.293290e+08,36.054662
...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Apply the function to each ticker and add the trailing P/E ratio as a new column to the DataFrame
fundamental_analysis_data["Share Trailing P/E"] = fundamental_analysis_data["Ticker"].apply(get_trailing_pe)


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [16]:
# Filter out NaN values
valid_values = fundamental_analysis_data["% Residual Enterprise Value"].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, "Ranking % Residual Enterprise Value"] = ranking.astype(int).astype(str)

In [17]:
# # Convert the "Share Trailing P/E" column to numeric, coercing errors to NaN
# fundamental_analysis_data["Share Trailing P/E"] = pd.to_numeric(fundamental_analysis_data["Share Trailing P/E"], errors='coerce')

# # Filter out NaN values
# valid_values = fundamental_analysis_data["Share Trailing P/E"].dropna()

# # Calculate rankings based on non-NaN values
# ranking = valid_values.rank(ascending=False, method='min')

# # Update the DataFrame with rankings
# fundamental_analysis_data.loc[valid_values.index, "Ranking Share Trailing P/E"] = ranking.astype(int).astype(str)

In [18]:
# # Sort the DataFrame by the "Ranking Share Trailing P/E" column in ascending order
# sorted_df = fundamental_analysis_data.sort_values(by="Ranking Share Trailing P/E", ascending=True)

# # Display the sorted DataFrame
# sorted_df

In [19]:
# Filter out rows with NaN or inf Share Trailing P/E values
revision = fundamental_analysis_data[['Ticker', 'Share Trailing P/E']].copy()
revision = revision.dropna()
revision = revision[~revision['Share Trailing P/E'].isin([np.inf, -np.inf])].copy()
revision

,Ticker,Share Trailing P/E
0,MSFT,37.307890
1,AAPL,29.591900
2,NVDA,62.117268
3,AMZN,50.488827
4,META,27.547234
...,...,...
2607,AGEN,1.174138
2631,CBAN,9.480000
2640,TCBX,10.480391
2654,EGRX,4.400000


In [20]:
# Convert the 'Share Trailing P/E' column to numeric, coercing errors to NaN
revision['Share Trailing P/E'] = pd.to_numeric(revision['Share Trailing P/E'], errors='coerce')

# Drop rows where 'Share Trailing P/E' is NaN (including those that were originally non-numeric or inf)
revision = revision.dropna(subset=['Share Trailing P/E'])

# Create the Rank column based on 'Share Trailing P/E' in ascending order
revision['Rank'] = revision['Share Trailing P/E'].rank(ascending=False, method='min').astype(int)

# Display the updated DataFrame
revision


,Ticker,Share Trailing P/E,Rank
0,MSFT,37.307890,403
1,AAPL,29.591900,566
2,NVDA,62.117268,196
3,AMZN,50.488827,239
4,META,27.547234,650
...,...,...,...
2607,AGEN,1.174138,1817
2631,CBAN,9.480000,1566
2640,TCBX,10.480391,1481
2654,EGRX,4.400000,1799


In [21]:
# Assuming `fundamental_analysis_data` and `revision` are already defined
# Ensure that the index of revision matches the index of fundamental_analysis_data

# Update the 'Ranking Share Trailing P/E' column in fundamental_analysis_data with ranks from revision
fundamental_analysis_data.loc[revision.index, 'Ranking Share Trailing P/E'] = revision['Rank'].values

# Display the updated DataFrame
fundamental_analysis_data


,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,37.307890,367,403.0
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,29.591900,358,566.0
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,62.117268,299,196.0
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,50.488827,515,239.0
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,27.547234,186,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:


# Assuming fundamental_analysis_data and revision are already defined
# Ensure that the index of revision matches the index of fundamental_analysis_data

# Update the 'Ranking Share Trailing P/E' column in fundamental_analysis_data with ranks from revision
fundamental_analysis_data.loc[revision.index, 'Ranking Share Trailing P/E'] = revision['Rank'].values

# Convert the 'Ranking Share Trailing P/E' column to numeric
fundamental_analysis_data['Ranking Share Trailing P/E'] = pd.to_numeric(fundamental_analysis_data['Ranking Share Trailing P/E'], errors='coerce')

# Sort the DataFrame by 'Ranking Share Trailing P/E'
sorted_fundamental_analysis_data = fundamental_analysis_data.sort_values(by='Ranking Share Trailing P/E', ascending=True)

# Display the sorted DataFrame
sorted_fundamental_analysis_data


,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E
1351,VCEL,VERICEL CORP,Cuidado de la Salud,Small-Mid,2.344060e+09,9.215300e+07,34653000.0,2.401560e+09,-5.750000e+07,-2.453009,4823.0,478,1.0
1555,INFA,INFORMATICA INC CLASS A,Tecnología de la Información,Small-Mid,8.959056e+09,1.879739e+09,328107008.0,1.051069e+10,-1.551632e+09,-17.319146,2990.0,1128,2.0
963,PRGO,PERRIGO PLC,Cuidado de la Salud,Small-Mid,3.800602e+09,4.249300e+09,384700000.0,7.665202e+09,-3.864600e+09,-101.683901,2788.0,2178,3.0
1368,SGRY,SURGERY PARTNERS INC,Cuidado de la Salud,Small-Mid,3.226917e+09,3.173600e+09,260000000.0,6.140517e+09,-2.913600e+09,-90.290527,2539.0,2134,4.0
1728,MODN,MODEL N INC,Tecnología de la Información,Small,1.176382e+09,2.900590e+08,60737000.0,1.405704e+09,-2.293220e+08,-19.493833,1492.0,1191,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Apply the function to each ticker and add the Forward P/E as a new column
fundamental_analysis_data['Forward P/E'] = fundamental_analysis_data['Ticker'].apply(get_forward_pe)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [24]:
# Display the updated DataFrame
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,37.307890,367,403.0,32.440422
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,29.591900,358,566.0,26.276625
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,62.117268,299,196.0,30.350338
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,50.488827,515,239.0,31.380207
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,27.547234,186,650.0,20.720104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Assuming you already have a DataFrame named fundamental_analysis_data with the "Forward P/E" column

# Drop rows with NaN values in the "Forward P/E" column
valid_values = fundamental_analysis_data['Forward P/E'].dropna()

# Calculate rankings based on non-NaN values in descending order
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Forward P/E'] = ranking.astype(int).astype(str)

# Display the updated DataFrame
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E,Ranking Forward P/E
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,37.307890,367,403.0,32.440422,287
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,29.591900,358,566.0,26.276625,424
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,62.117268,299,196.0,30.350338,324
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,50.488827,515,239.0,31.380207,306
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,27.547234,186,650.0,20.720104,675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Apply the function to each ticker and add the PEG ratio as a new column
fundamental_analysis_data['PEG Ratio (5 yr expected)'] = fundamental_analysis_data['Ticker'].apply(get_peg_ratio)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [27]:
# Assuming you already have a DataFrame named fundamental_analysis_data with the "Forward P/E" column

# Drop rows with NaN values in the "Forward P/E" column
valid_values = fundamental_analysis_data['PEG Ratio (5 yr expected)'].dropna()

# Calculate rankings based on non-NaN values in descending order
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking PEG Ratio (5 yr expected)'] = ranking.astype(int).astype(str)

# Replace non-finite valuefs with NaN
fundamental_analysis_data['Ranking PEG Ratio (5 yr expected)'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Convert the rank to integer type
fundamental_analysis_data['Ranking PEG Ratio (5 yr expected)'] = fundamental_analysis_data['Ranking PEG Ratio (5 yr expected)'].astype(float).astype(pd.Int64Dtype())

In [28]:
# Apply the function to each ticker and add the Price/Sales ratio as a new column
fundamental_analysis_data['Price/Sales'] = fundamental_analysis_data['Ticker'].apply(get_price_to_sales)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [29]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Price/Sales'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Price/Sales'] = ranking.astype(int)

In [30]:
fundamental_analysis_data.sort_values(by='Ranking Price/Sales')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E,Ranking Forward P/E,PEG Ratio (5 yr expected),Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales
2348,MBI,MBIA INC,Financieros,Micro,2.866356e+08,3.368000e+09,-1.890000e+08,3.843636e+09,-3.557000e+09,-1240.948230,NaN,2594,NaN,-7.453333,2393,-0.43,1889,-143.317830,1.0
2341,ORC,ORCHID ISLAND CAPITAL INC,Financieros,Small,4.661163e+08,3.711578e+09,5.564100e+07,4.122053e+09,-3.655937e+09,-784.340133,NaN,2578,NaN,61.214283,114,NaN,<NA>,-81.560160,2.0
592,NLY,ANNALY CAPITAL MANAGEMENT REIT INC,Financieros,Small-Mid,9.748571e+09,7.661462e+10,5.084064e+09,8.127912e+10,-7.153055e+10,-733.754212,NaN,2576,NaN,7.188191,2039,-1.46,2008,-42.608135,3.0
1912,CFFN,CAPITOL FEDERAL FINANCIAL INC,Financieros,Small,6.822527e+08,2.351022e+09,3.796500e+07,2.995310e+09,-2.313057e+09,-339.032291,NaN,2514,NaN,10.489796,1598,2.43,523,-16.070400,4.0
2194,TRTX,TPG RE FINANCE TRUST INC,Financieros,Small,6.747535e+08,2.657330e+09,9.402400e+07,3.238059e+09,-2.563306e+09,-379.887770,NaN,2526,NaN,7.688073,1983,1.69,823,-11.959473,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [31]:
# Apply the function to each ticker and add the Price to Book ratio as a new column
fundamental_analysis_data['Price/Book'] = fundamental_analysis_data['Ticker'].apply(get_price_to_book)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [32]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Price/Book'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Price/Book'] = ranking.astype(int)

In [33]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E,Ranking Forward P/E,PEG Ratio (5 yr expected),Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,...,367,403.0,32.440422,287,2.23,590,13.513519,2442.0,12.630219,2347.0
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,...,358,566.0,26.276625,424,2.70,441,7.633639,2253.0,39.276410,2467.0
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,...,299,196.0,30.350338,324,0.84,1382,32.829697,2523.0,53.341183,2477.0
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,...,515,239.0,31.380207,306,1.31,1065,3.184130,1636.0,8.678638,2230.0
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,...,186,650.0,20.720104,675,0.77,1417,8.499771,2295.0,8.113812,2202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN


In [34]:
# Apply the function to each ticker and add the trailing EPS as a new column
fundamental_analysis_data['Trailing EPS'] = fundamental_analysis_data['Ticker'].apply(get_trailing_eps)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [35]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Trailing EPS'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Trailing EPS'] = ranking.astype(int)

In [36]:
 # Apply another function to each ticker and add the forward EPS as a new column
fundamental_analysis_data['Forward EPS'] = fundamental_analysis_data['Ticker'].apply(get_forward_eps)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/METCV?modules=financialData%2CquoteType%2CdefaultKey

In [37]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Forward EPS'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Forward EPS'] = ranking.astype(int)

In [38]:
fundamental_analysis_data.sort_values(by='Ranking Forward EPS')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,PEG Ratio (5 yr expected),Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS
345,NVR,NVR INC,Consumo discrecional,Mid,2.323066e+10,1.024992e+09,1.328929e+09,2.292672e+10,3.039370e+08,1.308344,...,3.04,368,2.359142,1297.0,5.407471,2018.0,479.84,1.0,509.28,1.0
66,BKNG,BOOKING HOLDINGS INC,Consumo discrecional,Big,1.287667e+11,1.745600e+10,7.159000e+09,1.390637e+11,-1.029700e+10,-7.996631,...,0.97,1282,5.852501,2125.0,NaN,NaN,132.84,7.0,206.20,2.0
389,FCNCA,FIRST CITIZENS BANCSHARES INC CLAS,Financieros,Mid,2.562364e+10,3.974300e+10,3.483000e+09,6.188364e+10,-3.626000e+10,-141.509962,...,NaN,<NA>,2.701775,1442.0,1.234372,639.0,180.08,5.0,197.88,3.0
169,AZO,AUTOZONE INC,Consumo discrecional,Mid,4.788147e+10,1.230428e+10,3.057113e+09,5.712864e+10,-9.247166e+09,-19.312619,...,1.57,894,2.685360,1437.0,NaN,NaN,141.60,6.0,164.65,4.0
360,MKL,MARKEL GROUP INC,Financieros,Mid,2.104198e+10,3.855494e+09,3.133280e+09,2.176419e+10,-7.222139e+08,-3.432253,...,0.40,1626,1.265539,779.0,1.392262,755.0,185.13,4.0,100.39,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Apply the function to each ticker and add the current price as a new column
fundamental_analysis_data['Current Price'] = fundamental_analysis_data['Ticker'].apply(get_current_price)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=c.5l1UiRC69


Error fetching data for XTSLA: could not convert string to float: 'N/A'
Error fetching data for USD: could not convert string to float: 'N/A'
Error fetching data for HEIA: could not convert string to float: 'N/A'
Error fetching data for BFB: could not convert string to float: 'N/A'
Error fetching data for UHALB: could not convert string to float: 'N/A'


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFUT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFUT&crumb=c.5l1UiRC69


Error fetching data for MSFUT: could not convert string to float: 'N/A'
Error fetching data for MOGA: could not convert string to float: 'N/A'
Error fetching data for BFA: could not convert string to float: 'N/A'


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LENB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LENB&crumb=c.5l1UiRC69


...

Error fetching data for P5N994: could not convert string to float: 'N/A'


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RTYM4?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RTYM4&crumb=c.5l1UiRC69


Error fetching data for RTYM4: could not convert string to float: 'N/A'


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ESM4?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ESM4&crumb=c.5l1UiRC69


Error fetching data for ESM4: could not convert string to float: 'N/A'


In [40]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,...,590,13.513519,2442.0,12.630219,2347.0,11.53,167.0,13.26,184.0,430.16
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,...,441,7.633639,2253.0,39.276410,2467.0,6.42,394.0,7.23,461.0,189.98
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,...,1382,32.829697,2523.0,53.341183,2477.0,17.14,86.0,35.08,30.0,1064.69
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,...,1065,3.184130,1636.0,8.678638,2230.0,3.58,744.0,5.76,606.0,180.75
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,...,1417,8.499771,2295.0,8.113812,2202.0,17.36,80.0,23.08,61.0,478.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A


In [41]:
# Filter rows where Trailing EPS is positive
positive_trailing_eps = fundamental_analysis_data['Trailing EPS'] > 0

# Calculate Business Value only for rows with positive Trailing EPS
fundamental_analysis_data.loc[positive_trailing_eps, 'Trailing Business Value'] = (fundamental_analysis_data['Trailing EPS'] / 12) * fundamental_analysis_data['Current Price']

# For rows with negative or zero Trailing EPS, set Business Value to NaN
fundamental_analysis_data.loc[~positive_trailing_eps, 'Trailing Business Value'] = np.nan


In [42]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,...,13.513519,2442.0,12.630219,2347.0,11.53,167.0,13.26,184.0,430.16,413.312067
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,...,7.633639,2253.0,39.276410,2467.0,6.42,394.0,7.23,461.0,189.98,101.6393
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,...,32.829697,2523.0,53.341183,2477.0,17.14,86.0,35.08,30.0,1064.69,1520.732217
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,...,3.184130,1636.0,8.678638,2230.0,3.58,744.0,5.76,606.0,180.75,53.92375
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,...,8.499771,2295.0,8.113812,2202.0,17.36,80.0,23.08,61.0,478.22,691.824933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN


In [43]:
# Calculate % Deviation from the ideal price TEPS
fundamental_analysis_data['% Deviation Ideal Price TEPS'] = ((fundamental_analysis_data['Current Price'] / fundamental_analysis_data['Trailing Business Value']) -1) * 100

In [44]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['% Deviation Ideal Price TEPS'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking % Deviation Ideal Price TEPS'] = ranking.astype(int)

In [45]:
fundamental_analysis_data.sort_values(by='Ranking % Deviation Ideal Price TEPS')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS
345,NVR,NVR INC,Consumo discrecional,Mid,2.323066e+10,1.024992e+09,1.328929e+09,2.292672e+10,3.039370e+08,1.308344,...,5.407471,2018.0,479.84,1.0,509.28,1.0,7416.32,296553.915733,-97.499166,1.0
1790,SEB,SEABOARD CORP,Productos básicos de consumo,Small-Mid,3.253034e+09,1.811000e+09,6.360000e+08,4.428034e+09,-1.175000e+09,-36.120126,...,0.701690,169.0,238.52,2.0,NaN,NaN,3350.0,66586.833333,-94.968975,2.0
941,WTM,WHITE MOUNTAINS INSURANCE GROUP LT,Financieros,Small-Mid,4.439047e+09,5.321000e+08,3.550000e+08,4.616147e+09,-1.771000e+08,-3.989595,...,0.979626,403.0,221.11,3.0,60.00,9.0,1730.13,31879.087025,-94.572837,3.0
360,MKL,MARKEL GROUP INC,Financieros,Mid,2.104198e+10,3.855494e+09,3.133280e+09,2.176419e+10,-7.222139e+08,-3.432253,...,1.392262,755.0,185.13,4.0,100.39,5.0,1618.29,24966.168975,-93.518068,4.0
389,FCNCA,FIRST CITIZENS BANCSHARES INC CLAS,Financieros,Mid,2.562364e+10,3.974300e+10,3.483000e+09,6.188364e+10,-3.626000e+10,-141.509962,...,1.234372,639.0,180.08,5.0,197.88,3.0,1781.25,26730.625,-93.336295,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN


In [46]:
# Filter rows where Trailing EPS is positive
positive_forward_eps = fundamental_analysis_data['Forward EPS'] > 0

# Calculate Business Value only for rows with positive Trailing EPS
fundamental_analysis_data.loc[positive_forward_eps, 'Forward Business Value'] = (fundamental_analysis_data['Forward EPS'] / 12) * fundamental_analysis_data['Current Price']

# For rows with negative or zero Trailing EPS, set Business Value to NaN
fundamental_analysis_data.loc[~positive_forward_eps, 'Forward Business Value'] = np.nan

In [47]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,...,2347.0,11.53,167.0,13.26,184.0,430.16,413.312067,4.076323,167.0,475.3268
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,...,2467.0,6.42,394.0,7.23,461.0,189.98,101.6393,86.915888,394.0,114.46295
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,...,2477.0,17.14,86.0,35.08,30.0,1064.69,1520.732217,-29.988331,86.0,3112.443767
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,...,2230.0,3.58,744.0,5.76,606.0,180.75,53.92375,235.195531,744.0,86.76
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,...,2202.0,17.36,80.0,23.08,61.0,478.22,691.824933,-30.875576,80.0,919.776467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN


In [48]:
# Calculate % Deviation from the ideal price TEPS
fundamental_analysis_data['% Deviation Ideal Price FEPS'] = (fundamental_analysis_data['Forward Business Value'] / fundamental_analysis_data['Current Price']) - 1

In [49]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['% Deviation Ideal Price FEPS'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking % Deviation Ideal Price FEPS'] = ranking.astype(int)

In [50]:
fundamental_analysis_data.sort_values(by='Ranking % Deviation Ideal Price FEPS')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS
345,NVR,NVR INC,Consumo discrecional,Mid,2.323066e+10,1.024992e+09,1.328929e+09,2.292672e+10,3.039370e+08,1.308344,...,1.0,509.28,1.0,7416.32,296553.915733,-97.499166,1.0,314748.6208,41.44,1.0
66,BKNG,BOOKING HOLDINGS INC,Consumo discrecional,Big,1.287667e+11,1.745600e+10,7.159000e+09,1.390637e+11,-1.029700e+10,-7.996631,...,7.0,206.20,2.0,3795.35,42014.5245,-90.966576,7.0,65216.764167,16.183333,2.0
389,FCNCA,FIRST CITIZENS BANCSHARES INC CLAS,Financieros,Mid,2.562364e+10,3.974300e+10,3.483000e+09,6.188364e+10,-3.626000e+10,-141.509962,...,5.0,197.88,3.0,1781.25,26730.625,-93.336295,5.0,29372.8125,15.49,3.0
169,AZO,AUTOZONE INC,Consumo discrecional,Mid,4.788147e+10,1.230428e+10,3.057113e+09,5.712864e+10,-9.247166e+09,-19.312619,...,6.0,164.65,4.0,2792.9,32956.22,-91.525424,6.0,38320.915417,12.720833,4.0
360,MKL,MARKEL GROUP INC,Financieros,Mid,2.104198e+10,3.855494e+09,3.133280e+09,2.176419e+10,-7.222139e+08,-3.432253,...,4.0,100.39,5.0,1618.29,24966.168975,-93.518068,4.0,13538.344425,7.365833,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Add a new column to store the average growth rate of free cash flow
fundamental_analysis_data['Net Income Growth Rate 4 Yr'] = fundamental_analysis_data['Ticker'].apply(get_net_income_growth)

Error fetching data for XTSLA: 'Net Income'
Error fetching data for USD: 'Net Income'
Error fetching data for HEIA: 'Net Income'
Error fetching data for BFB: 'Net Income'
Error fetching data for UHALB: 'Net Income'
Error fetching data for MSFUT: 'Net Income'
Error fetching data for MOGA: 'Net Income'
Error fetching data for BFA: 'Net Income'
Error fetching data for LENB: 'Net Income'
Error fetching data for LGFB: 'Net Income'
Error fetching data for CWENA: 'Net Income'
Error fetching data for LGFA: 'Net Income'
Error fetching data for METCV: 'Net Income'
Error fetching data for GEFB: 'Net Income'
Error fetching data for ARD: 'Net Income'
Error fetching data for ADRO: 'Net Income'
Error fetching data for ADRO: 'Net Income'
Error fetching data for GTXI: 'Net Income'
Error fetching data for P5N994: 'Net Income'
Error fetching data for RTYM4: 'Net Income'
Error fetching data for ESM4: 'Net Income'


In [52]:
# Ensure 'Net Income Growth Rate 4 Yr' column is numeric, converting non-numeric entries to NaN
fundamental_analysis_data['Net Income Growth Rate 4 Yr'] = pd.to_numeric(
    fundamental_analysis_data['Net Income Growth Rate 4 Yr'], errors='coerce'
)

# Filter out non-finite values
valid_values = fundamental_analysis_data['Net Income Growth Rate 4 Yr'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Net Income Growth Rate 4 Yr'] = ranking.astype(int)

In [53]:
fundamental_analysis_data.sort_values(by='Ranking Net Income Growth Rate 4 Yr')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS,Net Income Growth Rate 4 Yr,Ranking Net Income Growth Rate 4 Yr
1320,MIR,MIRION TECHNOLOGIES INC CLASS A,Tecnología de la Información,Small-Mid,2.439669e+09,7.186000e+08,103900000.0,3.054369e+09,-6.147000e+08,-25.196038,...,1969.0,10.73,NaN,NaN,NaN,0.4292,-0.96,1969.0,4.423909e+06,1.0
617,WEX,WEX INC,Financieros,Small-Mid,7.867238e+09,4.194500e+09,727500032.0,1.133424e+10,-3.467000e+09,-44.068835,...,95.0,187.76,96.383467,94.805195,414.0,292.123267,0.555833,95.0,6.707744e+04,2.0
940,HOG,HARLEY DAVIDSON INC,Consumo discrecional,Small-Mid,4.735956e+09,7.278463e+09,812206976.0,1.120221e+10,-6.466256e+09,-136.535391,...,749.0,35.21,13.350458,163.736264,593.0,14.113342,-0.599167,749.0,1.666275e+04,3.0
916,ALTM,ARCADIUM LITHIUM PLC,Materiales,Small-Mid,5.050103e+09,6.384000e+08,141600000.0,5.546903e+09,-4.968000e+08,-9.837424,...,2043.0,4.7,NaN,NaN,NaN,0.133167,-0.971667,2043.0,1.508239e+04,4.0
777,SPXC,SPX TECHNOLOGIES INC,Industriales,Small-Mid,6.672925e+09,8.544000e+08,223400000.0,7.303925e+09,-6.310000e+08,-9.456124,...,579.0,144.34,39.813783,262.537764,800.0,72.17,-0.5,579.0,1.502658e+04,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Add a new column to store the average growth rate of free cash flow
fundamental_analysis_data['Free Cash Flow Growth Rate 4 Yr'] = fundamental_analysis_data['Ticker'].apply(get_free_cash_flow_growth)

Error fetching data for XTSLA: 'Free Cash Flow'
Error fetching data for USD: 'Free Cash Flow'
Error fetching data for HEIA: 'Free Cash Flow'
Error fetching data for BFB: 'Free Cash Flow'
Error fetching data for UHALB: 'Free Cash Flow'
Error fetching data for MSFUT: 'Free Cash Flow'
Error fetching data for MOGA: 'Free Cash Flow'
Error fetching data for BFA: 'Free Cash Flow'
Error fetching data for LENB: 'Free Cash Flow'
Error fetching data for LGFB: 'Free Cash Flow'
Error fetching data for CWENA: 'Free Cash Flow'
Error fetching data for LGFA: 'Free Cash Flow'
Error fetching data for METCV: 'Free Cash Flow'
Error fetching data for GEFB: 'Free Cash Flow'
Error fetching data for ARD: 'Free Cash Flow'
Error fetching data for ADRO: 'Free Cash Flow'
Error fetching data for ADRO: 'Free Cash Flow'
Error fetching data for GTXI: 'Free Cash Flow'
Error fetching data for P5N994: 'Free Cash Flow'
Error fetching data for RTYM4: 'Free Cash Flow'
Error fetching data for ESM4: 'Free Cash Flow'


In [55]:
# Ensure 'Net Income Growth Rate 4 Yr' column is numeric, converting non-numeric entries to NaN
fundamental_analysis_data['Free Cash Flow Growth Rate 4 Yr'] = pd.to_numeric(
    fundamental_analysis_data['Free Cash Flow Growth Rate 4 Yr'], errors='coerce'
)

# Filter out non-finite values
valid_values = fundamental_analysis_data['Free Cash Flow Growth Rate 4 Yr'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=False, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Free Cash Flow Growth Rate 4 Yr'] = ranking.astype(int)

In [56]:
fundamental_analysis_data

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS,Net Income Growth Rate 4 Yr,Ranking Net Income Growth Rate 4 Yr,Free Cash Flow Growth Rate 4 Yr,Ranking Free Cash Flow Growth Rate 4 Yr
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,...,413.312067,4.076323,167.0,475.3268,0.105,184.0,18.855173,1228.0,10.481719,1360.0
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,...,101.6393,86.915888,394.0,114.46295,-0.3975,461.0,22.504482,1148.0,11.983280,1317.0
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,...,1520.732217,-29.988331,86.0,3112.443767,1.923333,30.0,217.074971,212.0,209.884957,207.0
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,...,53.92375,235.195531,744.0,86.76,-0.52,606.0,-423.163981,2559.0,-144.267055,2282.0
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,...,691.824933,-30.875576,80.0,919.776467,0.923333,61.0,20.844183,1186.0,48.138900,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Add a new column to store the average growth rate of free cash flow
fundamental_analysis_data['Total Liabilities Growth Rate 4 Yr'] = fundamental_analysis_data['Ticker'].apply(get_total_liabilities_growth)

Error fetching data for XTSLA: 'Total Liabilities Net Minority Interest'
Error fetching data for USD: 'Total Liabilities Net Minority Interest'
Error fetching data for HEIA: 'Total Liabilities Net Minority Interest'
Error fetching data for BFB: 'Total Liabilities Net Minority Interest'
Error fetching data for UHALB: 'Total Liabilities Net Minority Interest'
Error fetching data for MSFUT: 'Total Liabilities Net Minority Interest'
Error fetching data for MOGA: 'Total Liabilities Net Minority Interest'
Error fetching data for BFA: 'Total Liabilities Net Minority Interest'
Error fetching data for LENB: 'Total Liabilities Net Minority Interest'
Error fetching data for LGFB: 'Total Liabilities Net Minority Interest'
Error fetching data for CWENA: 'Total Liabilities Net Minority Interest'
Error fetching data for LGFA: 'Total Liabilities Net Minority Interest'
Error fetching data for METCV: 'Total Liabilities Net Minority Interest'
Error fetching data for GEFB: 'Total Liabilities Net Minority 

In [58]:
# Ensure 'Net Income Growth Rate 4 Yr' column is numeric, converting non-numeric entries to NaN
fundamental_analysis_data['Total Liabilities Growth Rate 4 Yr'] = pd.to_numeric(
    fundamental_analysis_data['Total Liabilities Growth Rate 4 Yr'], errors='coerce'
)

# Filter out non-finite values
valid_values = fundamental_analysis_data['Total Liabilities Growth Rate 4 Yr'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Total Liabilities Growth Rate 4 Yr'] = ranking.astype(int)

In [59]:
fundamental_analysis_data.sort_values(by='Ranking Total Liabilities Growth Rate 4 Yr')

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,...,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS,Net Income Growth Rate 4 Yr,Ranking Net Income Growth Rate 4 Yr,Free Cash Flow Growth Rate 4 Yr,Ranking Free Cash Flow Growth Rate 4 Yr,Total Liabilities Growth Rate 4 Yr,Ranking Total Liabilities Growth Rate 4 Yr
1975,CGON,CG ONCOLOGY INC,Cuidado de la Salud,Small-Mid,2.179801e+09,404000.0,-63705000.0,2.243910e+09,-64109000.0,-2.941049,...,NaN,NaN,NaN,NaN,106.599325,374.0,85.017557,454.0,-56.714267,1.0
1284,DBRG,DIGITALBRIDGE GROUP INC CLASS A,Inmobiliario,Small-Mid,2.411493e+09,412577984.0,186894000.0,2.637177e+09,-225683984.0,-9.358682,...,1100.0,0.6912,-0.946667,1907.0,-80.738178,2186.0,56.967575,618.0,-47.399324,2.0
1639,LGND,LIGAND PHARMACEUTICALS INC,Cuidado de la Salud,Small,1.552064e+09,6194000.0,34361000.0,1.523897e+09,28167000.0,1.814809,...,486.0,39.888,-0.538333,642.0,-809.629937,2609.0,2.462645,1571.0,-46.698339,3.0
1960,TK,TEEKAY CORP,Energía,Small,8.946317e+08,79955000.0,608425984.0,3.661608e+08,528470984.0,59.071343,...,1221.0,0.350808,-0.964167,1996.0,295.719953,157.0,123.098595,333.0,-46.383950,4.0
1421,RXST,RXSIGHT INC,Cuidado de la Salud,Small-Mid,2.400131e+09,2522000.0,-33138000.0,2.435791e+09,-35660000.0,-1.485752,...,NaN,NaN,NaN,NaN,-88.880514,2212.0,10.219920,1372.0,-46.294856,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# import yfinance as yf
# import pandas as pd
# import numpy as np

# def get_average_stock_growth(ticker, years=5):
#     try:
#         # Fetch historical market data
#         company = yf.Ticker(ticker)
#         hist = company.history(period=f"{years + 1}y")  # Fetch history for one year more than required to calculate growth

#         # Ensure we have at least the required years of data
#         if len(hist) < (years + 1) * 252:  # Approx. 252 trading days in a year
#             print(f"Not enough data to calculate {years} years of growth.")
#             return "N/A"

#         # Extract adjusted closing prices


SyntaxError: incomplete input (<ipython-input-60-cf8ef2a0777e>, line 16)

In [61]:
pd.set_option('display.max_columns', None)
fundamental_analysis_data.head()

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E,Ranking Forward P/E,PEG Ratio (5 yr expected),Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS,Net Income Growth Rate 4 Yr,Ranking Net Income Growth Rate 4 Yr,Free Cash Flow Growth Rate 4 Yr,Ranking Free Cash Flow Growth Rate 4 Yr,Total Liabilities Growth Rate 4 Yr,Ranking Total Liabilities Growth Rate 4 Yr
0,MSFT,MICROSOFT CORP,Tecnología de la Información,Mega,3.197082e+12,1.062290e+11,1.101230e+11,3.193188e+12,3.894002e+09,0.121799,37.307890,367,403.0,32.440422,287,2.23,590,13.513519,2442.0,12.630219,2347.0,11.53,167.0,13.26,184.0,430.16,413.312067,4.076323,167.0,475.3268,0.105,184.0,18.855173,1228.0,10.481719,1360.0,3.984022,1051.0
1,AAPL,APPLE INC,Tecnología de la Información,Mega,2.913172e+12,1.045900e+11,1.105630e+11,2.907199e+12,5.973000e+09,0.205034,29.591900,358,566.0,26.276625,424,2.70,441,7.633639,2253.0,39.276410,2467.0,6.42,394.0,7.23,461.0,189.98,101.6393,86.915888,394.0,114.46295,-0.3975,461.0,22.504482,1148.0,11.983280,1317.0,4.141200,1068.0
2,NVDA,NVIDIA CORP,Tecnología de la Información,Mega,2.618956e+12,1.099100e+10,4.052400e+10,2.589423e+12,2.953300e+10,1.127663,62.117268,299,196.0,30.350338,324,0.84,1382,32.829697,2523.0,53.341183,2477.0,17.14,86.0,35.08,30.0,1064.69,1520.732217,-29.988331,86.0,3112.443767,1.923333,30.0,217.074971,212.0,209.884957,207.0,25.170481,2086.0
3,AMZN,AMAZON COM INC,Consumo discrecional,Mega,1.880993e+12,1.605600e+11,9.914700e+10,1.942406e+12,-6.141301e+10,-3.264925,50.488827,515,239.0,31.380207,306,1.31,1065,3.184130,1636.0,8.678638,2230.0,3.58,744.0,5.76,606.0,180.75,53.92375,235.195531,744.0,86.76,-0.52,606.0,-423.163981,2559.0,-144.267055,2282.0,13.014365,1688.0
4,META,META PLATFORMS INC CLASS A,Comunicación,Mega,1.213019e+12,3.763300e+10,7.636100e+10,1.174291e+12,3.872800e+10,3.192694,27.547234,186,650.0,20.720104,675,0.77,1417,8.499771,2295.0,8.113812,2202.0,17.36,80.0,23.08,61.0,478.22,691.824933,-30.875576,80.0,919.776467,0.923333,61.0,20.844183,1186.0,48.138900,703.0,35.293883,2230.0


**Rank by Non NAN's**

In [62]:
# Define the columns to sum
columns_to_sum = [
    "Ranking % Residual Enterprise Value",
    "Ranking Share Trailing P/E",
    "Ranking Forward P/E",
    "Ranking PEG Ratio (5 yr expected)",
    "Ranking Price/Sales",
    "Ranking Price/Book",
    "Ranking Trailing EPS",
    "Ranking Forward EPS",
    "Ranking % Deviation Ideal Price TEPS",
    "Ranking % Deviation Ideal Price FEPS",
    "Ranking Net Income Growth Rate 4 Yr",
    "Ranking Free Cash Flow Growth Rate 4 Yr",
    "Ranking Total Liabilities Growth Rate 4 Yr"
]

# Ensure all columns are numeric
for column in columns_to_sum:
    fundamental_analysis_data[column] = pd.to_numeric(fundamental_analysis_data[column], errors='coerce')

# Function to sum values if none are NaN
def sum_if_no_nan(row):
    if row[columns_to_sum].isnull().any():
        return np.nan
    else:
        return row[columns_to_sum].sum()

# Apply the function to each row
fundamental_analysis_data['Sum of Rankings'] = fundamental_analysis_data.apply(sum_if_no_nan, axis=1)

In [63]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Sum of Rankings'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Sum of Rankings'] = ranking.astype(int)

In [65]:
pd.set_option('display.max_columns', None)
fundamental_analysis_data.sort_values(by='Ranking Sum of Rankings', ascending=True)

,Ticker,Name,Sector,Market Cap Classification,Market Cap,Total Debt,Operating Cash Flow,Enterprise Value,Residual Enterprise Value,% Residual Enterprise Value,Share Trailing P/E,Ranking % Residual Enterprise Value,Ranking Share Trailing P/E,Forward P/E,Ranking Forward P/E,PEG Ratio (5 yr expected),Ranking PEG Ratio (5 yr expected),Price/Sales,Ranking Price/Sales,Price/Book,Ranking Price/Book,Trailing EPS,Ranking Trailing EPS,Forward EPS,Ranking Forward EPS,Current Price,Trailing Business Value,% Deviation Ideal Price TEPS,Ranking % Deviation Ideal Price TEPS,Forward Business Value,% Deviation Ideal Price FEPS,Ranking % Deviation Ideal Price FEPS,Net Income Growth Rate 4 Yr,Ranking Net Income Growth Rate 4 Yr,Free Cash Flow Growth Rate 4 Yr,Ranking Free Cash Flow Growth Rate 4 Yr,Total Liabilities Growth Rate 4 Yr,Ranking Total Liabilities Growth Rate 4 Yr,Sum of Rankings,Ranking Sum of Rankings
547,CHK,CHESAPEAKE ENERGY CORP,Energía,Mid,1.175763e+10,2.086000e+09,2.043000e+09,1.180063e+10,-4.299994e+07,-0.365720,11.962666,389.0,1382.0,15.712785,1032.0,8.36,111,2.427757,1331.0,1.099941,518.0,7.50,313.0,5.71,614.0,89.72,56.075,60.0,313.0,42.691767,-0.524167,614.0,952.896016,59.0,3394.711524,19.0,-26.301166,27.0,6722.0,1.0
419,RS,RELIANCE STEEL & ALUMINUM,Materiales,Mid,1.744620e+10,1.377000e+09,1.413000e+09,1.741020e+10,3.600000e+07,0.206349,14.163169,357.0,1254.0,15.757260,1023.0,6.97,136,1.204399,735.0,2.199330,1257.0,21.45,52.0,19.28,90.0,303.8,543.0425,-44.055944,52.0,488.105333,0.606667,90.0,95.216289,413.0,46.536267,724.0,-1.913254,553.0,6736.0,2.0
158,ROP,ROPER TECHNOLOGIES INC,Tecnología de la Información,Mid,5.830313e+10,7.722000e+09,2.102900e+09,6.392223e+10,-5.619100e+09,-9.637733,39.989720,825.0,359.0,27.452620,398.0,3.83,260,9.125834,2328.0,3.275874,1656.0,13.62,125.0,19.84,86.0,544.66,618.1891,-11.894273,125.0,900.504533,0.653333,86.0,82.040647,456.0,51.954453,662.0,-7.415441,251.0,7617.0,3.0
405,EME,EMCOR GROUP INC,Industriales,Mid,1.876430e+10,3.511120e+08,1.116502e+09,1.799891e+10,7.653900e+08,4.078968,26.316172,152.0,693.0,23.032352,550.0,24.07,39,1.429692,873.0,7.166816,2153.0,15.15,103.0,17.31,111.0,398.69,503.346125,-20.792079,103.0,575.110325,0.4425,111.0,83.415524,450.0,26.353239,981.0,11.278520,1597.0,7916.0,4.0
1394,ANDE,ANDERSONS INC,Productos básicos de consumo,Small,1.769568e+09,6.251620e+08,1.040658e+09,1.354072e+09,4.154960e+08,23.480079,14.680792,11.0,1231.0,14.436111,1154.0,2.22,593,0.130239,39.0,1.372435,735.0,3.54,749.0,3.60,966.0,51.97,15.33115,238.983051,749.0,15.591,-0.7,966.0,417.322838,129.0,67.828966,533.0,-8.049501,230.0,8085.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,ADRO,CHINOOK THERAPEUTICS INC CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2682,GTXI,GTXI INC - CVR,Cuidado de la Salud,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2683,P5N994,Petrocorp Inc Escrow,Energía,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,RTYM4,RUSSELL 2000 EMINI CME JUN 24,Liquidez,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Rank by Only at Leat 1 NAN**


In [ ]:
import numpy as np

# Define the columns to sum
columns_to_sum = [
    "Ranking % Residual Enterprise Value",
    "Ranking Share Trailing P/E",
    "Ranking Forward P/E",
    "Ranking PEG Ratio (5 yr expected)",
    "Ranking Price/Sales",
    "Ranking Price/Book",
    "Ranking Trailing EPS",
    "Ranking Forward EPS",
    "Ranking % Deviation Ideal Price TEPS",
    "Ranking % Deviation Ideal Price FEPS",
    "Ranking Net Income Growth Rate 4 Yr",
    "Ranking Free Cash Flow Growth Rate 4 Yr",
    "Ranking Total Liabilities Growth Rate 4 Yr"
]

# Ensure all columns are numeric
for column in columns_to_sum:
    fundamental_analysis_data[column] = pd.to_numeric(fundamental_analysis_data[column], errors='coerce')

# Function to sum values if at least one is NaN
def sum_if_any_nan(row):
    if row[columns_to_sum].notnull().any():
        return row[columns_to_sum].sum()
    else:
        return np.nan

# Apply the function to each row
fundamental_analysis_data['Sum of Rankings with NAN'] = fundamental_analysis_data.apply(sum_if_any_nan, axis=1)

In [ ]:
# Filter out non-finite values
valid_values = fundamental_analysis_data['Sum of Rankings with NAN'].dropna()

# Calculate rankings based on non-NaN values
ranking = valid_values.rank(ascending=True, method='min')

# Update the DataFrame with rankings
fundamental_analysis_data.loc[valid_values.index, 'Ranking Sum of Rankings with NAN'] = ranking.astype(int)

In [ ]:
fundamental_analysis_data_order_by_best_NAN = fundamental_analysis_data.sort_values(by='Ranking Sum of Rankings with NAN')
fundamental_analysis_data_order_by_best_NAN

In [66]:
#fundamental_analysis_data_order_by_best = fundamental_analysis_data.sort_values(by='Sum of Rankings', ascending=True)
# Assuming fundamental_data_analysis DataFrame is defined earlier
# Convert DataFrame to CSV
fundamental_analysis_data.to_csv('fundamental_data_analysis.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('fundamental_data_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import yfinance as yf

# Fetch information about Meta Platforms, Inc. (META)
meta = yf.Ticker("NVDA")

# Get statistics
statistics = meta.info

# Print all statistics
print("Statistics of Meta Platforms, Inc. (META):")
for key, value in statistics.items():
    print(key, ":", value)
